# 미니프로젝트

1. bike data 불러오기

- 각 데이터 확인(shape, info, ...)


2. 중복 확인

- 훈련 데이터, 테스트 데이터 중복 값 있는지 확인

- 훈련 데이터, 테스트 데이터 null 값 확인

3. datetime에서 년,월,일,시,분,초를 추출하여 각각의 column으로 구성

ex) df['year]=df['datetime'].dt.year

-년,월,일,시 에 따른 평균 대여량 구하기

=> 분석 결과 reporting (ex. 여름에 대여량이 많다)

4. 근무일 유무/요일/시즌/날씨에 따른 시간대별 자전거 대여량 구하기

5. 연도별 월별 자전거 대여량 구하기

6. 아웃라이어제거

정상범위 데이터 : count열값-count열값평균 < 3*(count.std)

7. 시즌컬럼 원핫 인코딩

-season_1, ... season_4

0 0 0 1

...

-기존 데이터 프레임에 원핫인코딩 결과열 결합

8.데이터 보정

-풍속 등 각 필드에 0으로 저장되어 있는 값을 근사값으로 보정

9. 바이크 대여수 예상되는 값 출력

-조건문 기반 작성

10. 제출

In [134]:
'''
데이터 설명
매달 1~19일 : train
매달 19~말일 : test
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather -   1: Clear, Few clouds, Partly cloudy, Partly cloudy
            2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
            3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
            4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - 실제 온도
atemp - 체감 온도
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
'''

'\n데이터 설명\n매달 1~19일 : train\n매달 19~말일 : test\ndatetime - hourly date + timestamp  \nseason -  1 = spring, 2 = summer, 3 = fall, 4 = winter \nholiday - whether the day is considered a holiday\nworkingday - whether the day is neither a weekend nor holiday\nweather -   1: Clear, Few clouds, Partly cloudy, Partly cloudy\n            2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist\n            3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds\n            4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog \ntemp - 실제 온도\natemp - 체감 온도\nhumidity - relative humidity\nwindspeed - wind speed\ncasual - number of non-registered user rentals initiated\nregistered - number of registered user rentals initiated\ncount - number of total rentals\n'

In [135]:
import pandas as pd
import numpy as np

from numpy import NaN, nan, NAN
from pandas import DataFrame

import warnings
warnings.filterwarnings(action='ignore')

In [136]:
train = pd.read_csv('train.csv', parse_dates=['datetime'])
test = pd.read_csv('test.csv', parse_dates=['datetime'])

trainCopy = train.copy()
testCopy = test.copy()

In [137]:
# 데이터 형태 파악
trainCopy.shape  # (10886, 12)
trainCopy.describe()
# trainCopy.info()  # 10886행

testCopy.shape  # (6493, 9)
testCopy.describe()
# testCopy.info()  # 6493행
trainCopy.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [138]:
# 중복된 행이 있는지 탐색
trainCopy[trainCopy.duplicated()]  # False
testCopy[testCopy.duplicated()]  # False

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed


In [139]:
# null을 포함한 행 확인
trainCopy.isnull().sum()
testCopy.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64

In [140]:
# 날짜요소 추출 후 새로운 열 생성

trainCopy['year'] = trainCopy.datetime.dt.year
trainCopy['month'] = trainCopy.datetime.dt.month
trainCopy['day'] = trainCopy.datetime.dt.day
trainCopy['hour'] = trainCopy.datetime.dt.hour
trainCopy['minute'] = trainCopy.datetime.dt.minute
trainCopy['second'] = trainCopy.datetime.dt.second
trainCopy['dayofweek'] = trainCopy.datetime.dt.dayofweek

testCopy['year'] = testCopy.datetime.dt.year
testCopy['month'] = testCopy.datetime.dt.month
testCopy['day'] = testCopy.datetime.dt.day
testCopy['hour'] = testCopy.datetime.dt.hour
testCopy['minute'] = testCopy.datetime.dt.minute
testCopy['second'] = testCopy.datetime.dt.second
testCopy['dayofweek'] = testCopy.datetime.dt.dayofweek

In [141]:
trainCopy = trainCopy.drop('datetime', axis=1)
testCopy = testCopy.drop('datetime', axis=1)

In [142]:
# 데이터 표준화
from sklearn.preprocessing import MinMaxScaler

trainCopy[['temp','atemp','humidity']]=DataFrame(MinMaxScaler().fit_transform(trainCopy[['temp','atemp','humidity']])
          ,columns=['temp','atemp','humidity'])

testCopy[['temp','atemp','humidity']]=DataFrame(MinMaxScaler().fit_transform(testCopy[['temp','atemp','humidity']])
          ,columns=['temp','atemp','humidity'])

In [143]:
# 데이터프래임 열 순서 재정렬
trainCopy = trainCopy.iloc[:,[11,12,13,14,15,16,17,0,1,2,3,4,5,6,7,8,9,10]]
testCopy = testCopy.iloc[:,[8,9,10,11,12,13,14,0,1,2,3,4,5,6,7]]

In [144]:
# 각 상활별 대여량 평균

trainCopy.groupby(['year'])['count'].mean()
# 2011    144.223349
# 2012    238.560944

trainCopy.groupby(['month'])['count'].mean()
# 1      90.366516  MIN
# 2     110.003330
# 3     148.169811
# 4     184.160616
# 5     219.459430
# 6     242.031798
# 7     235.325658  MAX
# 8     234.118421
# 9     233.805281
# 10    227.699232
# 11    193.677278
# 12    175.614035

trainCopy.groupby(['day'])['count'].mean()
# 1     180.333913
# 2     183.910995
# 3     194.696335
# 4     195.705575
# 5     189.765217
# 6     189.860140
# 7     183.773519
# 8     179.041812  MIN
# 9     187.897391
# 10    195.183566
# 11    195.679577
# 12    190.675393
# 13    194.160279
# 14    195.829268
# 15    201.527875
# 16    191.353659
# 17    205.660870  MAX
# 18    192.605684
# 19    192.311847

trainCopy.groupby(['hour'])['count'].mean()
# 0      55.138462
# 1      33.859031
# 2      22.899554
# 3      11.757506
# 4       6.407240  MIN
# 5      19.767699
# 6      76.259341
# 7     213.116484
# 8     362.769231
# 9     221.780220
# 10    175.092308
# 11    210.674725
# 12    256.508772
# 13    257.787281
# 14    243.442982
# 15    254.298246
# 16    316.372807
# 17    468.765351  MAX
# 18    430.859649
# 19    315.278509
# 20    228.517544
# 21    173.370614
# 22    133.576754
# 23     89.508772

trainCopy.groupby(['season'])['count'].mean()
# 1 봄      116.343261  MIN
# 2 여름    215.251372
# 3 가을    234.417124  MAX
# 4 겨울    198.988296

trainCopy.groupby(['weather'])['count'].mean()
# 0.000000    205.236791  좋은날      MAX
# 0.333333    178.955540  구름낀날
# 0.666667    118.846333  비좀오는날   MIN
# 1.000000    164.000000  안 좋은날


# # 7월, 17시, 가을, 좋은날에 가장 많이 탐

weather
1    205.236791
2    178.955540
3    118.846333
4    164.000000
Name: count, dtype: float64

In [145]:
# 피크타임 확인

trainCopy.groupby(['holiday','hour'])['count'].mean()[0] # 17~18시
trainCopy.groupby(['holiday','hour'])['count'].mean()[1] # 12~18시
trainCopy.groupby(['season','hour'])['count'].mean()
trainCopy.groupby(['weather','hour'])['count'].mean()
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[0] # 월요일 17~18시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[1] # 화요일 17~18시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[2] # 수요일 17~18시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[3] # 목요일 17~18시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[4] # 금요일 17~18시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[5] # 토요일 12~16시
trainCopy.groupby(['dayofweek','hour'])['count'].mean()[6] # 일요일 12~16시

hour
0      96.227273
1      79.454545
2      62.484375
3      30.393939
4       9.651515
5       9.476190
6      15.136364
7      34.742424
8      83.954545
9     158.666667
10    264.636364
11    321.242424
12    381.318182
13    384.848485
14    370.409091
15    364.515152
16    365.530303
17    326.287879
18    273.469697
19    227.257576
20    172.590909
21    131.939394
22     99.787879
23     64.757576
Name: count, dtype: float64

In [146]:
# 년도별 월별 대여 총량

trainCopy.groupby(['year'])['count'].sum()
# 2011     781979
# 2012    1303497

trainCopy.groupby(['month'])['count'].sum()
# 1      79884
# 2      99113
# 3     133501
# 4     167402
# 5     200147
# 6     220733
# 7     214617
# 8     213516
# 9     212529
# 10    207434
# 11    176440
# 12    160160

month
1      79884
2      99113
3     133501
4     167402
5     200147
6     220733
7     214617
8     213516
9     212529
10    207434
11    176440
12    160160
Name: count, dtype: int64

In [147]:
# count값의 아웃라이어 제거  (# 정상범위 데이터 : count열값-count열값평균 < 3*(count.std))
# 10886 -> 10739로 행 감소
trainCopy=trainCopy[trainCopy['count']-trainCopy['count'].mean() < 3*(trainCopy['count'].std())]

In [148]:
trainCopy.shape  # (10739, 18) 이상치가 제거된 값

(10739, 18)

In [149]:
# 원핫인코딩 열 추가
trainCopy = pd.concat([trainCopy,pd.get_dummies(trainCopy['season'],prefix='season')],axis=1)
# trainCopy = trainCopy.drop('season',axis=1)

testCopy = pd.concat([testCopy,pd.get_dummies(testCopy['season'],prefix='season')],axis=1)
# testCopy = testCopy.drop('season',axis=1)

In [150]:
# trainCopy 결측값 처리
# windspeed가 0이 아닌 애들
x_train=trainCopy[['season','weather','hour','temp']][trainCopy['windspeed']!=0]
x_train_label=trainCopy[['windspeed']][trainCopy['windspeed']!=0]
x_train_label
x_train
x_train_label.windspeed

# windspeed가 0인 애들
x_test=trainCopy[['season','weather','hour','temp']][trainCopy['windspeed']==0]
x_test_label=trainCopy[['windspeed']][trainCopy['windspeed']==0]
x_test
x_test_label

# testCopy 결측값 처리
y_train=testCopy[['season','weather','hour','temp']][testCopy['windspeed']!=0]
y_train_label=testCopy[['windspeed']][testCopy['windspeed']!=0]
y_train_label
y_train
y_train_label.windspeed

# windspeed가 0인 애들
y_test=testCopy[['season','weather','hour','temp']][testCopy['windspeed']==0]
y_test_label=testCopy[['windspeed']][testCopy['windspeed']==0]
y_test
y_test_label

# knn을 이용해 예상값 입력
from sklearn.neighbors import KNeighborsRegressor

clf = KNeighborsRegressor(n_neighbors=2)
clf.fit(x_train,x_train_label)

trainCopy.loc[trainCopy['windspeed']==0,'windspeed'] = clf.predict(x_test)

clf.fit(y_train,y_train_label)
testCopy.loc[testCopy['windspeed']==0,'windspeed'] = clf.predict(y_test)

In [151]:
trainCopy.columns

Index(['year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek',
       'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'casual', 'registered', 'count', 'season_1',
       'season_2', 'season_3', 'season_4'],
      dtype='object')

In [152]:
# 학습시킬 데이터셋 만들기
train_test=trainCopy.loc[:,['month','hour','holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed','season_1','season_2', 'season_3', 'season_4']]
# label데이터셋
train_label=trainCopy.loc[:,['count']]
train_test # (10739, 14)

# 테스트할 데이터셋 만들기
test_test=testCopy.loc[:,['month','hour','holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed','season_1','season_2', 'season_3', 'season_4']]
train_test # (6493, 14)

,month,hour,holiday,workingday,weather,temp,atemp,humidity,windspeed,season_1,season_2,season_3,season_4
0,1,0,0,0,1,0.224490,0.305068,0.81,23.00110,1,0,0,0
1,1,1,0,0,1,0.204082,0.288064,0.80,9.50060,1,0,0,0
2,1,2,0,0,1,0.204082,0.288064,0.80,10.99805,1,0,0,0
3,1,3,0,0,1,0.224490,0.305068,0.75,7.50065,1,0,0,0
4,1,4,0,0,1,0.224490,0.305068,0.75,11.00140,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,12,19,0,1,1,0.367347,0.423649,0.50,26.00270,0,0,0,1
10882,12,20,0,1,1,0.346939,0.372860,0.57,15.00130,0,0,0,1
10883,12,21,0,1,1,0.326531,0.338964,0.61,15.00130,0,0,0,1
10884,12,22,0,1,1,0.326531,0.372860,0.61,6.00320,0,0,0,1


In [153]:
# KNN 학습기 학습시키기
clf.fit(train_test,train_label)
res = clf.predict(test_test)
res = res.astype(int)
res

array([[ 31],
       [  1],
       [  4],
       ...,
       [155],
       [ 93],
       [ 70]])

In [154]:
sub = pd.read_csv('sampleSubmission.csv')
sub.loc[:,'count'] = res
sub.to_csv('sub11.csv', index=False)
sub

,datetime,count
0,2011-01-20 00:00:00,31
1,2011-01-20 01:00:00,1
2,2011-01-20 02:00:00,4
3,2011-01-20 03:00:00,4
4,2011-01-20 04:00:00,3
...,...,...
6488,2012-12-31 19:00:00,299
6489,2012-12-31 20:00:00,131
6490,2012-12-31 21:00:00,155
6491,2012-12-31 22:00:00,93
